In [ ]:
##step0 : dcm2niix

In [ ]:
#!/bin/bash

####################################################################################################################
##########NOW that the proper inputs (extracted inputs) are made, let's change them into NIfTI and do BIDS###########
#####################################################################################################################

#MUST BE DONE ON NODE1(with no conda activation)! (NODE2에서 dwMRI conda activate해서 하면 이상하게 FSGPR이 안나옴... 
year=19
input_dir=/storage/bigdata/CHA_bigdata/1.raw_MRI/20${year}_MRI
output_dir=/storage/bigdata/CHA_bigdata/2.dcm2niix/20${year}_MRI_BIDS #output directory

mkdir $output_dir

for sub in $input_dir/*         #possible because we already made subject folders in the output
do
    sub_idx="${year}${sub:(-4)}"
    mkdir "${output_dir}/sub-${sub_idx}" #sub_idx : subject number #extracts the last ten (ex : 2019-C0018 from the whole /storage/bigdata/CHA_bigdata/2019-C0018 ))

#####################
    echo $sub			# this sub folder has the absolute path for each stuff!

    sub_out="${output_dir}/sub-${sub_idx}" #output directory for this specific subject
    mkdir "${sub_out}/anat"
    mkdir "${sub_out}/dwi"
    mkdir "${sub_out}/tmp"
######################

    echo `dcm2niix -b y -ba n -o ${sub_out}/tmp -z y "${sub}/"` #save to the sub_out folder

    for files in $sub_out/tmp/* #do DICOM to niix for EACH SUBJECT
    do
    echo $files
###################### anat ######################

    # && : if the one in the front is successful, proceed to the next
    # conditional statement [ $a == $b ] requires spaces between all stuff

    # t1
    if [[ "${files}" == *"FSPGR"* ]] && [[ "${files}" == *".json"* ]]
    then
        mv $files "${sub_out}/anat/sub-${sub_idx}_T1w.json"

    elif [[ "${files}" == *"FSPGR"* ]] && [[ "${files}" == *".nii.gz"* ]]
    then
        mv $files "${sub_out}/anat/sub-${sub_idx}_T1w.nii.gz"

###################### dwi ######################
    ################multishell####### (move the previous things to the tmp folder thing)
    elif [[ "${files}" == *"DTI"* ]] && [[ "${files}" == *".json"* ]]
    then
        mv $files "${sub_out}/dwi/sub-${sub_idx}_dwi.json"

    elif [[ "${files}" == *"DTI"* ]] && [[ "${files}" == *".nii.gz"* ]]
    then
        mv $files "${sub_out}/dwi/sub-${sub_idx}_dwi.nii.gz"

    elif [[ "${files}" == *"bval" ]]
    then
        mv $files "${sub_out}/dwi/sub-${sub_idx}_dwi.bval"

    elif [[ "${files}" == *"bvec" ]]
    then
        mv $files "${sub_out}/dwi/sub-${sub_idx}_dwi.bvec"
    fi
    done

rm -r ${sub_out}/tmp

done


In [ ]:
#! /bin/bash

year=21
input_dir=/temp_storage/bigdata/CHA_bigdata/test_one_sub/2.dcm2niix/20${year}_MRI_BIDS
save_dir=/temp_storage/bigdata/CHA_bigdata/test_one_sub/save_dir
json_dir=/temp_storage/bigdata/CHA_bigdata/2.dcm2niix/dataset_description.json

echo $files_dir

#mkdir save_dir

for sub in $input_dir/sub-*
do
    echo $sub
    sub_name="${sub:(-10)}"
    one_BIDS=$save_dir/$sub_name
    mkdir $one_BIDS
    cp $json_dir $one_BIDS
    cp -r $sub $one_BIDS


done

In [ ]:
#! /bin/bash

#SBATCH --job-name one_BIDS_ALL  #job name을 다르게 하기 위해서
#SBATCH --nodes=1
#SBATCH --nodelist=node1 #used node2
#SBATCH -t 800:00:00 # Time for running job #길게 10일넘게 잡음
#SBATCH -o ./step1_qsiprep_results/output_%A-%a.out #%J : job id 가 들어가는 것
#SBATCH -e ./step1_qsiprep_results/error_%A-%a.error
#SBATCH --ntasks=1
#SBATCH --mem-per-cpu=1GB
#SBATCH --cpus-per-task=1
#SBATCH --array=1-100%15


# uses older versino of qsiprep because new versino of qsiprep doesn't have reconall, and doing reconall seperately results in dwi T1 registration errors.... 

#sub_list=`ls ./save_dir` #이것하면 다 나오지 않아서 그냥 sublist를 cat하기로함

sub=`cat sub_list.txt | awk -v var=${SLURM_ARRAY_TASK_ID} 'NR==var'`

#total 235, but will only do the first 10

echo $sub
mkdir ./3.qsiprep_results/$sub

docker run --rm --name SUB_${SLURM_ARRAY_TASK_ID}_node1_upto100 \
-v /storage/bigdata/CHA_bigdata/test_one_sub/save_dir/$sub/:/data:ro \
-v /storage/bigdata/CHA_bigdata/test_one_sub/3.qsiprep_results/$sub/:/out \
-v /storage/bigdata/CHA_bigdata:/freesurfer \
pennbbl/qsiprep:0.14.3 \
/data \
/out \
participant \
-w /out/tmp \
--output_resolution 1.2 \
--denoise_after_combining \
--unringing_method mrdegibbs \
--b0_to_t1w_transform Affine \
--intramodal_template_transform SyN \
--do_reconall \
--fs-license-file /freesurfer/license.txt \
--skip_bids_validation \
--nthreads 1 \
--infant \

In [ ]:
#!/bin/bash

#SBATCH --job-name ONE_RECON__only #job name을 다르게 하기 위해서
#SBATCH --nodes=1
#SBATCH --nodelist=node3
#SBATCH -t 800:00:00 # Time for running job
#SBATCH -o ./step2_with_reconstruction/output_%A-%a.out #%J : job id 가 들어가는 것
#SBATCH -e ./step2_with_reconstruction/error_%A-%a.error
#SBATCH --ntasks=1
#SBATCH --mem-per-cpu=1GB
#SBATCH --cpus-per-task=4
#SBATCH --array=212-236%8

sub=`cat sub_list.txt | awk -v var=${SLURM_ARRAY_TASK_ID} 'NR==var'`

#total 235, but will only do the first 10

echo $sub


docker run --rm -v /scratch/connectome/dyhan316/dwMRI/license.txt:/opt/freesurfer/license.txt:ro\
 -v /storage/bigdata/CHA_bigdata/test_one_sub/save_dir/$sub/:/data:ro\
 -v /storage/bigdata/CHA_bigdata/test_one_sub/3.qsiprep_results/$sub/:/qsiprep-output:ro\
 -v /storage/bigdata/CHA_bigdata/test_one_sub/3.qsiprep_results/$sub//freesurfer:/sngl/freesurfer-input:ro\
 -v /storage/bigdata/CHA_bigdata/test_one_sub/3.qsiprep_results/$sub/:/out pennbbl/qsiprep:0.15.4\
 /data /out participant --recon-input /qsiprep-output --freesurfer-input /sngl/freesurfer-input\
 --recon-spec mrtrix_multishell_msmt_ACT-hsvs --output_resolution 1.2 --denoise_after_combining\
 --unringing_method mrdegibbs --b0_to_t1w_transform Affine --intramodal_template_transform SyN\
 --nthreads 4 --skip_bids_validation --recon_only --infant

In [ ]:
##++++BACKUP###


In [ ]:
#!/bin/bash

####################################################################################################################
##########NOW that the proper inputs (extracted inputs) are made, let's change them into NIfTI and do BIDS###########
#####################################################################################################################

#MUST BE DONE ON NODE1(with no conda activation)! (NODE2에서 dwMRI conda activate해서 하면 이상하게 FSGPR이 안나옴... 
year=19
input_dir=/storage/bigdata/CHA_bigdata/1.raw_MRI/20${year}_MRI
output_dir=/storage/bigdata/CHA_bigdata/2.dcm2niix/20${year}_MRI_BIDS #output directory

mkdir $output_dir

for sub in $input_dir/*         #possible because we already made subject folders in the output
do
    sub_idx="${year}${sub:(-4)}"
    mkdir "${output_dir}/sub-${sub_idx}" #sub_idx : subject number #extracts the last ten (ex : 2019-C0018 from the whole /storage/bigdata/CHA_bigdata/2019-C0018 ))

#####################
    echo $sub			# this sub folder has the absolute path for each stuff!

    sub_out="${output_dir}/sub-${sub_idx}" #output directory for this specific subject
    mkdir "${sub_out}/anat"
    mkdir "${sub_out}/dwi"
    mkdir "${sub_out}/tmp"
######################

    echo `dcm2niix -b y -ba n -o ${sub_out}/tmp -z y "${sub}/"` #save to the sub_out folder

    for files in $sub_out/tmp/* #do DICOM to niix for EACH SUBJECT
    do
    echo $files
###################### anat ######################

    # && : if the one in the front is successful, proceed to the next
    # conditional statement [ $a == $b ] requires spaces between all stuff

    # t1
    if [[ "${files}" == *"FSPGR"* ]] && [[ "${files}" == *".json"* ]]
    then
        mv $files "${sub_out}/anat/sub-${sub_idx}_T1w.json"

    elif [[ "${files}" == *"FSPGR"* ]] && [[ "${files}" == *".nii.gz"* ]]
    then
        mv $files "${sub_out}/anat/sub-${sub_idx}_T1w.nii.gz"

###################### dwi ######################
    ################multishell####### (move the previous things to the tmp folder thing)
    elif [[ "${files}" == *"DTI"* ]] && [[ "${files}" == *".json"* ]]
    then
        mv $files "${sub_out}/dwi/sub-${sub_idx}_dwi.json"

    elif [[ "${files}" == *"DTI"* ]] && [[ "${files}" == *".nii.gz"* ]]
    then
        mv $files "${sub_out}/dwi/sub-${sub_idx}_dwi.nii.gz"

    elif [[ "${files}" == *"bval" ]]
    then
        mv $files "${sub_out}/dwi/sub-${sub_idx}_dwi.bval"

    elif [[ "${files}" == *"bvec" ]]
    then
        mv $files "${sub_out}/dwi/sub-${sub_idx}_dwi.bvec"
    fi
    done

rm -r ${sub_out}/tmp

done


In [ ]:
#! /bin/bash

year=21
input_dir=/temp_storage/bigdata/CHA_bigdata/test_one_sub/2.dcm2niix/20${year}_MRI_BIDS
save_dir=/temp_storage/bigdata/CHA_bigdata/test_one_sub/save_dir
json_dir=/temp_storage/bigdata/CHA_bigdata/2.dcm2niix/dataset_description.json

echo $files_dir

#mkdir save_dir

for sub in $input_dir/sub-*
do
    echo $sub
    sub_name="${sub:(-10)}"
    one_BIDS=$save_dir/$sub_name
    mkdir $one_BIDS
    cp $json_dir $one_BIDS
    cp -r $sub $one_BIDS


done

In [ ]:
#! /bin/bash

#SBATCH --job-name one_BIDS_ALL  #job name을 다르게 하기 위해서
#SBATCH --nodes=1
#SBATCH --nodelist=node1 #used node2
#SBATCH -t 800:00:00 # Time for running job #길게 10일넘게 잡음
#SBATCH -o ./step1_qsiprep_results/output_%A-%a.out #%J : job id 가 들어가는 것
#SBATCH -e ./step1_qsiprep_results/error_%A-%a.error
#SBATCH --ntasks=1
#SBATCH --mem-per-cpu=1GB
#SBATCH --cpus-per-task=1
#SBATCH --array=1-100%15


# uses older versino of qsiprep because new versino of qsiprep doesn't have reconall, and doing reconall seperately results in dwi T1 registration errors.... 

#sub_list=`ls ./save_dir` #이것하면 다 나오지 않아서 그냥 sublist를 cat하기로함

sub=`cat sub_list.txt | awk -v var=${SLURM_ARRAY_TASK_ID} 'NR==var'`

#total 235, but will only do the first 10

echo $sub
mkdir ./3.qsiprep_results/$sub

docker run --rm --name SUB_${SLURM_ARRAY_TASK_ID}_node1_upto100 \
-v /storage/bigdata/CHA_bigdata/test_one_sub/save_dir/$sub/:/data:ro \
-v /storage/bigdata/CHA_bigdata/test_one_sub/3.qsiprep_results/$sub/:/out \
-v /storage/bigdata/CHA_bigdata:/freesurfer \
pennbbl/qsiprep:0.14.3 \
/data \
/out \
participant \
-w /out/tmp \
--output_resolution 1.2 \
--denoise_after_combining \
--unringing_method mrdegibbs \
--b0_to_t1w_transform Affine \
--intramodal_template_transform SyN \
--do_reconall \
--fs-license-file /freesurfer/license.txt \
--skip_bids_validation \
--nthreads 1 \
--infant \

In [ ]:
#!/bin/bash

#SBATCH --job-name ONE_RECON__only #job name을 다르게 하기 위해서
#SBATCH --nodes=1
#SBATCH --nodelist=node3
#SBATCH -t 800:00:00 # Time for running job
#SBATCH -o ./step2_with_reconstruction/output_%A-%a.out #%J : job id 가 들어가는 것
#SBATCH -e ./step2_with_reconstruction/error_%A-%a.error
#SBATCH --ntasks=1
#SBATCH --mem-per-cpu=1GB
#SBATCH --cpus-per-task=4
#SBATCH --array=212-236%8

sub=`cat sub_list.txt | awk -v var=${SLURM_ARRAY_TASK_ID} 'NR==var'`

#total 235, but will only do the first 10

echo $sub


docker run --rm -v /scratch/connectome/dyhan316/dwMRI/license.txt:/opt/freesurfer/license.txt:ro\
 -v /storage/bigdata/CHA_bigdata/test_one_sub/save_dir/$sub/:/data:ro\
 -v /storage/bigdata/CHA_bigdata/test_one_sub/3.qsiprep_results/$sub/:/qsiprep-output:ro\
 -v /storage/bigdata/CHA_bigdata/test_one_sub/3.qsiprep_results/$sub//freesurfer:/sngl/freesurfer-input:ro\
 -v /storage/bigdata/CHA_bigdata/test_one_sub/3.qsiprep_results/$sub/:/out pennbbl/qsiprep:0.15.4\
 /data /out participant --recon-input /qsiprep-output --freesurfer-input /sngl/freesurfer-input\
 --recon-spec mrtrix_multishell_msmt_ACT-hsvs --output_resolution 1.2 --denoise_after_combining\
 --unringing_method mrdegibbs --b0_to_t1w_transform Affine --intramodal_template_transform SyN\
 --nthreads 4 --skip_bids_validation --recon_only --infant